In [2]:
# Install necessary libraries
!pip3 install transformers
!pip3 install scipy


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 44.1/44.1 MB 1.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
import torch
import re

In [4]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
def get_bert_embeddings(text):
    # Tokenize input text and get BERT embeddings
    input_ids = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

In [6]:
def clean_text(text):
    # Basic text cleaning
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

In [7]:
def calculate_similarity(embeddings1, embeddings2):
    # Calculate cosine similarity between two sets of embeddings
    return 1 - cosine(embeddings1, embeddings2)

In [8]:
def are_similar(article1, article2, threshold=0.8):
    # Check if two articles are similar based on BERT embeddings
    cleaned_article1 = clean_text(article1)
    cleaned_article2 = clean_text(article2)

    embeddings1 = get_bert_embeddings(cleaned_article1)
    embeddings2 = get_bert_embeddings(cleaned_article2)

    similarity = calculate_similarity(embeddings1, embeddings2)

    return similarity >= threshold

In [12]:
# Example usage:
article1 = "The water supply will be suspended in parts of Colombo from 7 p.m. tomorrow (04) until 5 a.m. on Sunday (05) the National Water Supply & Drainage Board (NWSDB) says.As such, Colombo 11, 12, 13, 14 and 15 will be affected by the 10-hour water cut.The Water Board attributed the water cut to the essential maintenance work planned under the Ambatale Water Supply System Improvements and Energy Saving Project."
article2 = "Mr. B.L.A.J. Dharmakeerthi has been appointed as the new Secretary to the Ministry of Agriculture and Plantation Industries.The appointment was made by President Ranil Wickremesinghe, with effect from October 27, 2023.This was communicated in a special gazette notification published by Presidential Secretary Saman Ekanayake on November 02."

if are_similar(article1, article2):
    print("The articles have similar content.")
else:
    print("The articles have different content.")

The articles have different content.
